In [46]:
import nest_asyncio
nest_asyncio.apply()

In [47]:
import pandas as pd
import asyncio 
import numpy as np
from binance import BinanceSocketManager
from binance.client import Client
import BinanceKeys

In [48]:
client = Client(BinanceKeys.api_key, BinanceKeys.api_secret)

In [49]:
ST = 7
LT = 25

In [50]:
def gethistorical(symbol, long_term):
    df = pd.DataFrame(client.get_historical_klines(symbol, '1d', str(long_term) + 'days ago',
                                                   '1 day ago'))
    closes = pd.DataFrame(df[4])
    closes.columns = ['Close']
    closes['ST'] = closes.Close.rolling(ST-1).sum()
    closes['LT'] = closes.Close.rolling(LT-1).sum()
    closes.dropna(inplace=True)
    return closes

In [51]:
historicals = gethistorical('MKRUSDT', LT)

In [52]:
historicals

,Close,ST,LT
23,1379.00000000,7968.0,33054.0


In [53]:
def liveSMA(hist, live):
    liveST = (hist['ST'].values + live.Price.values) / ST
    liveLT = (hist['LT'].values + live.Price.values) / LT
    return liveST, liveLT

In [54]:
def createframe(msg):  # Data cleaning
    df = pd.DataFrame([msg])
    df = df.loc[:, ['s', 'E', 'p']]
    df.columns = ['currency', 'Time', 'Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [55]:
async def main(coin, qty, SL_limit, open_position=False):
    bm = BinanceSocketManager(client)
    ts = bm.trade_socket(coin)
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            if res:
                frame = createframe(res)
                print(frame)
                livest, livelt = liveSMA(historicals, frame)
                if livest > livelt and not open_position:
                    print("Order Completed")
                    buyprice = frame.Price[0]
                    open_position = True
                if open_position:
                    if frame.Price[0] < buyprice * SL_limit or frame.Price[0] > 1.02 * buyprice:
                        print("Sold")

In [ ]:
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main('MKRUSDT', 100, 0.98))

  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:23.167  1380.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:36.205  1380.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:42.829  1380.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:42.829  1380.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:43.168  1380.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:50.654  1379.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:50.654  1379.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:55.121  1379.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:55.121  1379.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:18:55.158  1379.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:10.521  1380.0
  currency                    Time   Price
0  MKRUSDT 

KeyboardInterrupt: 

  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:31.044  1381.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:31.044  1381.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:38.469  1381.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:38.469  1381.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:38.469  1381.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:19:55.195  1381.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:20:01.623  1382.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:20:01.623  1382.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:20:01.795  1382.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:20:01.795  1382.0
  currency                    Time   Price
0  MKRUSDT 2023-11-14 14:20:05.486  1382.0
  currency                    Time   Price
0  MKRUSDT 